# Урок 7. Сверточные нейронные сети для анализа текста
## Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
  - Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
  - Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=c7d11f2d4840f4430e80fbef99353de2e91dd0d785a5a6f14bf7959f42f92b3e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 8.2 MB 5.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=73957b0988e882d0126f8a66678d4c88cbb4d562cdaf1505b0bcec6169471c27
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [3]:
! pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 27.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [4]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
data =pd.read_excel('отзывы за лето.xls')
data

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


Проведем препроцессинг текста:

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)
    
data['Content'] = data['Content'].apply(preprocess_text)
data

,Rating,Content,Date
0,5,it just works,2017-08-14
1,4,целое удобноной приложениеиз минус хотеть боль...,2017-08-14
2,5,отлично,2017-08-14
3,5,зависать 1 работа антивирус ранее пользоваться...,2017-08-14
4,5,удобно работать быстро,2017-08-14
...,...,...,...
20654,1,шляпас роот право бесполезный прогаразрабыв ох...,2017-06-01
20655,5,около,2017-06-01
20656,4,довольный,2017-06-01
20657,1,песопаснастя рута нужын,2017-06-01


In [7]:
data.Rating.value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Посмотрим максимальное количество токенов в отзывах:

In [8]:
maxlen = data.Content.apply(lambda x: len(x.split())).max()
maxlen

132

Разобьем выборку на тренировочную и тестовую:

In [86]:
X_train, X_test, train_y, test_y = train_test_split(data.Content, data.Rating, test_size=0.33, random_state=42)

Приступим к составлению словаря:

In [10]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [13]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

Определим размер словаря:

In [14]:
max_words = len(dist) + 1
max_words

9976

In [15]:
tokens_filtered = list(dist.keys())

In [16]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered, 1)).items()}

In [17]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

Преобразуем текст в индексы с паддингами:

In [18]:
x_train = np.asarray([text_to_sequence(text, maxlen) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, maxlen) for text in X_test], dtype=np.int32)

In [19]:
x_train.shape

(13841, 132)

In [20]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 
import tensorflow as tf

Преобразуем таргет как категориальную переменную:

In [87]:
y_train = pd.DataFrame(train_y)
y_test = pd.DataFrame(test_y)

In [88]:
num_classes = 6
y_train = tf.keras.utils.to_categorical(y_train['Rating'], num_classes)
y_test = tf.keras.utils.to_categorical(y_test['Rating'], num_classes)

## Сеть с эмбеддингом по умолчанию.
Определим архитектуру сети. Берем выходную размерность эмбеддинга равную 300, так как это наиболее частая размерность предобученных векторов в RusVectores.

In [109]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=300, input_length=maxlen))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [110]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 132, 300)          2992800   
                                                                 
 conv1d_13 (Conv1D)          (None, 128, 128)          192128    
                                                                 
 activation_35 (Activation)  (None, 128, 128)          0         
                                                                 
 global_max_pooling1d_10 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_22 (Dense)            (None, 100)               12900     
                                                                 
 activation_36 (Activation)  (None, 100)               0         
                                                     

In [111]:
batch_size = 512
epochs = 5

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
25/25 [==============================] - 1s 40ms/step - loss: 1.1669 - accuracy: 0.6773 - val_loss: 0.9380 - val_accuracy: 0.7047
Epoch 2/5
25/25 [==============================] - 1s 31ms/step - loss: 0.7818 - accuracy: 0.7437 - val_loss: 0.7301 - val_accuracy: 0.7451
Epoch 3/5
25/25 [==============================] - 1s 31ms/step - loss: 0.6276 - accuracy: 0.7827 - val_loss: 0.6861 - val_accuracy: 0.7509
Epoch 4/5
25/25 [==============================] - 1s 33ms/step - loss: 0.5399 - accuracy: 0.8097 - val_loss: 0.6813 - val_accuracy: 0.7581
Epoch 5/5
25/25 [==============================] - 1s 31ms/step - loss: 0.4632 - accuracy: 0.8424 - val_loss: 0.6959 - val_accuracy: 0.7661


In [112]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report


class Metrics(Callback):


    def on_test_end(self, epoch, logs=None):
        val_predict = model.predict(x_test).argmax(axis=1)
        print('\n', classification_report(test_y, val_predict))

In [113]:
model.evaluate(x_test, y_test, callbacks=[Metrics()])

211/214 [============================>.] - ETA: 0s - loss: 0.6658 - accuracy: 0.7774
               precision    recall  f1-score   support

           1       0.54      0.67      0.60       753
           2       0.00      0.00      0.00       240
           3       0.29      0.12      0.17       301
           4       0.31      0.21      0.25       666
           5       0.87      0.95      0.91      4858

    accuracy                           0.78      6818
   macro avg       0.40      0.39      0.38      6818
weighted avg       0.72      0.78      0.75      6818

214/214 [==============================] - 1s 5ms/step - loss: 0.6660 - accuracy: 0.7774


[0.6660494208335876, 0.7773540616035461]

Как видно по отчету классификации, сеть очень плохо справляется с классификацией со всеми классами, кроме 5-го.

Посмотрим на веса эмбеддинга:

In [55]:
model.layers[0].weights

[<tf.Variable 'embedding_8/embeddings:0' shape=(9976, 300) dtype=float32, numpy=
 array([[ 4.32159267e-02, -2.78614443e-02,  5.38497083e-02, ...,
          1.65194459e-03,  5.65565526e-02,  5.55325560e-02],
        [-2.78971121e-02, -1.28846625e-02,  2.17216946e-02, ...,
          7.10575655e-02, -5.41978404e-02, -3.01585905e-02],
        [ 1.45514170e-03, -2.65098475e-02,  6.55869842e-02, ...,
         -8.93394556e-03, -2.35939771e-02, -1.13569840e-05],
        ...,
        [ 4.42090146e-02, -1.15358345e-02, -3.11907660e-02, ...,
         -6.30980730e-03,  1.83698647e-02, -3.79914641e-02],
        [ 4.76066954e-02,  1.34612434e-02, -1.42255649e-02, ...,
         -4.54571620e-02,  3.41218151e-02,  2.97570266e-02],
        [ 3.25902216e-02, -7.13188574e-03, -8.18797201e-03, ...,
         -4.20645848e-02, -3.18241492e-02,  4.34644856e-02]], dtype=float32)>]

## Сеть с предобученными весами эмбеддинга
Скачаем модель эмбеддинга, самую легковесную в списке:

In [56]:
from gensim.models import KeyedVectors

ru_w2v = KeyedVectors.load_word2vec_format('https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz', binary=True)

Следующие 6 ячеек - код из туториала по обработке текста перед применением моделей векторизации RusVectores. Дело в том, что почти все представленные модели принимают на вход слова в формате 'слово_POS-TAG'. 

In [57]:
! pip install ufal.udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 304 kB 33.2 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626601 sha256=06f303b42f2b6cdf65fd38816b5efd1d82684fa1e9573456eeb28d560171be7f
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe


In [58]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=4afc86eb37d454dc92c70b1e455c84cb488378d6b2fd1b6e720cca32641656b7
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [59]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'

modelfile = wget.download(udpipe_url)

In [60]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [61]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


In [62]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    # print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    # print('Processing input...', file=sys.stderr)
    # for line in text:
    #     # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
    #     output = process(process_pipeline, text=line)
    #     print(' '.join(output))
    return process(process_pipeline, text=text)

Проверим работу функции тегирования:

In [63]:
tag_ud(text='однажды')

['однажды_ADV']

In [64]:
vocabulary

{'наконецтый': 1,
 'исправить': 2,
 'чушь': 3,
 'снеоргинальный': 4,
 'прошивка': 5,
 'приложение': 6,
 'удобно': 7,
 'пользоваться': 8,
 'использование': 9,
 'отлично': 10,
 'класс': 11,
 'замечательный': 12,
 'довольный': 13,
 'влиять': 14,
 'работа': 15,
 'устройство': 16,
 'приходиться': 17,
 'периодически': 18,
 'отключать': 19,
 'оплатить': 20,
 'мобильный': 21,
 'коммунальный': 22,
 'услуга': 23,
 'гораздо': 24,
 'простой': 25,
 'ноут': 26,
 'шаблон': 27,
 'проводиться': 28,
 'клик': 29,
 'сказать': 30,
 'пропасть': 31,
 'переводить': 32,
 'другой': 33,
 'клиент': 34,
 'сбербанк': 35,
 'оплачивать': 36,
 'чужой': 37,
 'телефон': 38,
 'программа': 39,
 'серьёзно': 40,
 'поработать': 41,
 'смело': 42,
 'качать': 43,
 'устраивать': 44,
 'принцип': 45,
 'хороший': 46,
 'быстро': 47,
 'обновление': 48,
 'сталон': 49,
 'делать': 50,
 'выписка': 51,
 'счёт': 52,
 'тк': 53,
 'увидеть': 54,
 'поступление': 55,
 'сегодняшний': 56,
 'число': 57,
 'происходить': 58,
 'день': 59,
 'просить':

In [65]:
tag_vocab = tag_ud(' '.join(vocabulary.keys()))

Обработанный список из ключей словаря:

In [66]:
tag_vocab

['наконецвать_VERB',
 'исправлять_VERB',
 'чушь_NOUN',
 'снеоргинальный_ADJ',
 'прошивка_NOUN',
 'приложение_NOUN',
 'удобный_ADJ',
 'пользоваться_VERB',
 'использование_NOUN',
 'отлично_ADV',
 'класс_NOUN',
 'замечательный_ADJ',
 'довольньый_ADJ',
 'влиять_VERB',
 'работа_NOUN',
 'устройство_NOUN',
 'приходиться_VERB',
 'периодически_ADV',
 'отключать_VERB',
 'оплачивать_VERB',
 'мобильный_ADJ',
 'коммунальный_ADJ',
 'услуга_NOUN',
 'гораздо_ADV',
 'простой_ADJ',
 'ноать_NOUN',
 'шаблон_NOUN',
 'проводить_VERB',
 'клик_NOUN',
 'сказать_VERB',
 'пропасть_NOUN',
 'переводить_VERB',
 'другой_ADJ',
 'клиент_NOUN',
 'сбербанк_NOUN',
 'оплачивать_VERB',
 'чужой_ADJ',
 'телефон_NOUN',
 'программа_NOUN',
 'серьезно_ADV',
 'поработать_VERB',
 'смело_ADV',
 'качать_VERB',
 'устраивать_VERB',
 'принцип_NOUN',
 'хороший_ADJ',
 'быстро_ADV',
 'обновление_NOUN',
 'талоть_ADJ',
 'делать_VERB',
 'выписка_NOUN',
 'счет_VERB',
 'тк_NOUN',
 'видеть_VERB',
 'поступление_NOUN',
 'сегодняшний_ADJ',
 'число

Составим по словарю список векторов, первым вектором добавив нулевой вектор паддинга. Также нулевыми векторами будут все слова, которых нет в словаре энкодера.

In [67]:
zero_point = np.zeros(300) # вектор для паддинга и неизвестных слов
weights_w2v = []
weights_w2v.append(zero_point) # добавляем паддинг
for itm in tag_vocab:
    try:
        word = ru_w2v[itm]
    except:
        word = zero_point
    weights_w2v.append(word)
weights_w2v = np.array(weights_w2v)

Посмотрим на получившиеся вектора. Они и будут весами эмбеддинга:

In [68]:
weights_w2v

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.02837435, -0.00300759, -0.02165741, ...,  0.05755383,
         0.0651065 , -0.04093882],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.12418263,  0.0542766 , -0.02237881, ..., -0.06872032,
         0.0387696 , -0.0385087 ]])

Размерность весов эмбединга и получившихся векторов совпадает.

In [69]:
weights_w2v.shape

(9976, 300)

Скомпилируем новую сеть с архитектурой, полностью повторящей предыдущую:

In [114]:
model_w2v = Sequential()
model_w2v.add(Embedding(input_dim=max_words, output_dim=300, input_length=maxlen))
model_w2v.add(Conv1D(128, 5))
model_w2v.add(Activation("relu"))
model_w2v.add(GlobalMaxPool1D())
model_w2v.add(Dense(100))
model_w2v.add(Activation("relu"))
model_w2v.add(Dense(num_classes))
model_w2v.add(Activation('softmax'))

In [115]:
model_w2v.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_w2v.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 132, 300)          2992800   
                                                                 
 conv1d_14 (Conv1D)          (None, 128, 128)          192128    
                                                                 
 activation_38 (Activation)  (None, 128, 128)          0         
                                                                 
 global_max_pooling1d_11 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_24 (Dense)            (None, 100)               12900     
                                                                 
 activation_39 (Activation)  (None, 100)               0         
                                                     

Посмотрим на веса эмбединга:

In [116]:
model_w2v.layers[0].get_weights()

[array([[ 0.02241856, -0.03046808, -0.00917349, ...,  0.03809459,
         -0.00862749, -0.00885659],
        [ 0.0328071 ,  0.00269746, -0.03280994, ...,  0.01143064,
          0.01101186,  0.02155768],
        [-0.0482181 ,  0.04544935, -0.0243639 , ..., -0.00495157,
         -0.03340178,  0.00195251],
        ...,
        [ 0.01684235, -0.03749365,  0.04496777, ...,  0.01591004,
          0.00748163, -0.04722345],
        [-0.04769764, -0.02435048, -0.02206792, ..., -0.02017682,
         -0.02364231,  0.00645015],
        [-0.02346289,  0.00278411,  0.00268685, ...,  0.0018787 ,
         -0.04808731, -0.02277663]], dtype=float32)]

Установим веса эмбединга ранее полученными по словарю векторами:

In [117]:
model_w2v.layers[0].set_weights([weights_w2v])

In [118]:
model_w2v.layers[0].get_weights()

[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.02837435, -0.00300759, -0.02165741, ...,  0.05755383,
          0.0651065 , -0.04093882],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.12418263,  0.0542766 , -0.02237881, ..., -0.06872032,
          0.0387696 , -0.0385087 ]], dtype=float32)]

Заморозим слой эмбединга:

In [119]:
model_w2v.layers[0].trainable = False

Посмотрим на модель. Обучаемых параметров действительно меньше, чем общих

In [120]:
model_w2v.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 132, 300)          2992800   
                                                                 
 conv1d_14 (Conv1D)          (None, 128, 128)          192128    
                                                                 
 activation_38 (Activation)  (None, 128, 128)          0         
                                                                 
 global_max_pooling1d_11 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_24 (Dense)            (None, 100)               12900     
                                                                 
 activation_39 (Activation)  (None, 100)               0         
                                                     

Обучим сеть:

In [121]:
batch_size = 512
epochs = 5

history = model_w2v.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/5
25/25 [==============================] - 2s 41ms/step - loss: 1.2142 - accuracy: 0.6749 - val_loss: 0.8498 - val_accuracy: 0.7300
Epoch 2/5
25/25 [==============================] - 1s 31ms/step - loss: 0.7221 - accuracy: 0.7598 - val_loss: 0.7024 - val_accuracy: 0.7487
Epoch 3/5
25/25 [==============================] - 1s 31ms/step - loss: 0.6055 - accuracy: 0.7842 - val_loss: 0.6783 - val_accuracy: 0.7574
Epoch 4/5
25/25 [==============================] - 1s 32ms/step - loss: 0.5311 - accuracy: 0.8111 - val_loss: 0.6800 - val_accuracy: 0.7552
Epoch 5/5
25/25 [==============================] - 1s 32ms/step - loss: 0.4560 - accuracy: 0.8399 - val_loss: 0.6898 - val_accuracy: 0.7661


In [122]:
class Metrics2(Callback):


    def on_test_end(self, epoch, logs=None):
        val_predict = model_w2v.predict(x_test).argmax(axis=1)
        print('\n', classification_report(test_y, val_predict))

Проведем валидацию:

In [123]:
model_w2v.evaluate(x_test, y_test, callbacks=[Metrics2()])

200/214 [===========================>..] - ETA: 0s - loss: 0.6596 - accuracy: 0.7761
               precision    recall  f1-score   support

           1       0.53      0.71      0.61       753
           2       0.00      0.00      0.00       240
           3       0.25      0.17      0.20       301
           4       0.31      0.16      0.21       666
           5       0.88      0.95      0.91      4858

    accuracy                           0.78      6818
   macro avg       0.39      0.40      0.39      6818
weighted avg       0.72      0.78      0.75      6818

214/214 [==============================] - 1s 5ms/step - loss: 0.6532 - accuracy: 0.7769


[0.6532443761825562, 0.7769140601158142]

**Выводы**
- Обе сетки работают примерно одинаково. Одинаково плохо)
- Качество предобученного эмбединга и обученного также, получается, примерно одинаково, хотя предобученный сильно разряжен нулевыми векторами